# Prepare my own spreadsheet for 2017 Yahoo Fantasy Baseball (roto format)
The goal is to identify players who could be over-/under-valued based on their performance in 2016. For hitters and pitchers, I identify stats that I think either (1) reflect their true talent, or (2) are heavily influenced by luck. For each stat, I calculate the Z-score (the number of standard deviations away from the average). For each player, I sum the Z-scores from all the stats that I chose. The players are then ranked based on their total Z-scores.
This is meant to be a guide for me when bidding for players. If a player has a high total Z-score, I could go a bit higher in bidding; if a player has a low total Z-score, I would refrain myself from overpaying.
This ranking ignores all the projection systems and is based on performance in 2016 only. Therefore, it is not a strict ranking of players, but rather as an aid for deciding how much to pay for a given player.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline

### Define functions that clean up hitter data

In [2]:
RATE_COLUMNS = ['BB%', 'K%', 'O-Swing%', 'Z-Contact%', 'Contact%', 'SwStr%', 'HR/FB']
P_RATE_COLUMNS = ['LOB%', 'GB%', 'HR/FB', 'O-Swing%', 'Contact%', 'F-Strike%', 'SwStr%', 'K%', 'BB%']

def reformat_rate_columns(df, columns=RATE_COLUMNS):
    """
    Re-format rate stats from percentages to decimal numbers.
    """
    for col in columns:
        values = df[col].str.replace('%', '')
        values = values.astype('float') / 100.
        df[col] = values
    return df


def calc_Zscore_hitter(df, col, positive=True, min_PA=50):
    """
    Calculate Z scores for a given column, using players with at least min_PA plate appearances.
    The argument "positive" means higher values for this column is more desirable.
    """
    qual = (df['PA'] >= min_PA)
    values = df.loc[qual, col]
    norm, scale = sm.robust.scale.huber(values)
    zscore = (df[col] - norm) / scale
    if not positive:
        zscore = zscore * -1
    df = df.assign(**{'zscore_'+col: zscore})
    return df


def calc_Zscore_pitcher(df, col, positive=True, min_IP=50):
    """
    Calculate Z scores for a given column, using players with at least min_PA plate appearances.
    The argument "positive" means higher values for this column is more desirable.
    """
    qual = (df['IP'] >= min_IP)
    values = df.loc[qual, col]
    norm, scale = sm.robust.scale.huber(values)
    zscore = (df[col] - norm) / scale
    if not positive:
        zscore = zscore * -1
    df = df.assign(**{'zscore_'+col: zscore})
    return df

In [15]:
positions = ['C', '1B', '2B', '3B', 'SS', 'OF']
hitters = {}
for pos in positions:
    df = pd.read_csv('{}_2017.csv'.format(pos), index_col='playerid')
    df = reformat_rate_columns(df)
    df = calc_Zscore_hitter(df, 'BB%')
    df = calc_Zscore_hitter(df, 'K%', positive=False)
    df = calc_Zscore_hitter(df, 'GB/FB', positive=False)
    df = calc_Zscore_hitter(df, 'HR/FB', positive=False)
    df = calc_Zscore_hitter(df, 'O-Swing%', positive=False)
    df = calc_Zscore_hitter(df, 'Z-Contact%')
    df = calc_Zscore_hitter(df, 'SwStr%', positive=False)
    zscore_col = [col for col in df.columns if col.startswith('zscore')]
    df['zscore_sum'] = df[zscore_col].sum(axis=1) / 7.
    columns = ['Name', 'WAR', 'zscore_sum']
    for col in df.columns:
        if col not in columns + ['Fantasy', '$']:
            columns.append(col)
    hitters[pos] = df[columns].sort('WAR', ascending=False)

/Users/khuang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:19: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [16]:
hitters['C'].head()

,Name,WAR,zscore_sum,Team,G,PA,HR,R,RBI,SB,...,Contact%,SwStr%,HR/FB,zscore_BB%,zscore_K%,zscore_GB/FB,zscore_HR/FB,zscore_O-Swing%,zscore_Z-Contact%,zscore_SwStr%
playerid,,,,,,,,,,,,,,,,,,,,,
7870,Jonathan Lucroy,4.5,0.248643,- - -,142,544,24,67,81,5,...,0.818,0.078,0.158,0.092654,0.629172,0.607008,-0.919283,-0.070199,0.474547,0.926602
9166,Buster Posey,4.0,0.624590,Giants,146,614,14,82,80,6,...,0.870,0.061,0.098,0.602243,1.858438,-0.806740,0.029565,0.012299,1.136286,1.540043
1433,Wilson Ramos,3.5,-0.300039,Nationals,131,523,22,58,80,0,...,0.819,0.091,0.214,-0.445244,1.184868,-1.903977,-1.804875,-0.103199,0.514653,0.457500
11739,J.T. Realmuto,3.5,-0.000542,Marlins,137,545,11,60,48,12,...,0.824,0.085,0.088,-0.898211,0.646012,-0.785640,0.187707,-0.482691,0.655021,0.674009
11442,Gary Sanchez,3.2,-0.954577,Yankees,53,229,20,34,42,1,...,0.710,0.130,0.400,0.630553,-0.465379,-0.405827,-4.746305,-0.598188,-0.147086,-0.949805


In [17]:
for pos in hitters.keys():
    hitters[pos].to_csv('{}_2017_zscore.csv'.format(pos), index=False)

## Pitchters

In [13]:
sp = pd.read_csv('SP_2017.csv', index_col='playerid')
sp = reformat_rate_columns(sp, columns=P_RATE_COLUMNS)
sp = calc_Zscore_pitcher(sp, 'BABIP')
sp = calc_Zscore_pitcher(sp, 'LOB%', positive=False)
sp = calc_Zscore_pitcher(sp, 'GB/FB')
sp = calc_Zscore_pitcher(sp, 'HR/FB')
sp = calc_Zscore_pitcher(sp, 'O-Swing%')
sp = calc_Zscore_pitcher(sp, 'Contact%', positive=False)
sp = calc_Zscore_pitcher(sp, 'F-Strike%')
sp = calc_Zscore_pitcher(sp, 'SwStr%')
sp = calc_Zscore_pitcher(sp, 'K%')
sp = calc_Zscore_pitcher(sp, 'BB%')

zscore_columns = [col for col in sp.columns if col.startswith('zscore')]
sp = sp.assign(**{'zscore_sum': sp[zscore_columns].sum(axis=1) / 10.})
columns = ['Name', 'zscore_sum']
for col in sp.columns:
    if col not in columns + ['Fantasy', '$']:
        columns.append(col)

sp[columns].sort('WAR', ascending=False).to_csv('SP_2017_zscore.csv', index=False)

/Users/khuang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [14]:
rp = pd.read_csv('RP_2017.csv', index_col='playerid')
rp = reformat_rate_columns(rp, columns=P_RATE_COLUMNS)
rp = calc_Zscore_pitcher(rp, 'BABIP')
rp = calc_Zscore_pitcher(rp, 'LOB%', positive=False)
rp = calc_Zscore_pitcher(rp, 'GB/FB')
rp = calc_Zscore_pitcher(rp, 'HR/FB')
rp = calc_Zscore_pitcher(rp, 'O-Swing%')
rp = calc_Zscore_pitcher(rp, 'Contact%', positive=False)
rp = calc_Zscore_pitcher(rp, 'F-Strike%')
rp = calc_Zscore_pitcher(rp, 'SwStr%')
rp = calc_Zscore_pitcher(rp, 'K%')
rp = calc_Zscore_pitcher(rp, 'BB%')

zscore_columns = [col for col in rp.columns if col.startswith('zscore')]
rp = rp.assign(**{'zscore_sum': rp[zscore_columns].sum(axis=1) / 10.})
columns = ['Name', 'zscore_sum']
for col in rp.columns:
    if col not in columns + ['Fantasy', '$']:
        columns.append(col)

rp[columns].sort('WAR', ascending=False).to_csv('RP_2017_zscore.csv', index=False)

/Users/khuang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
